**Notice**: this notebook has been used to write the code easily. The notebook has then be converted into a file, "../src/generate_comparison_data.py", that has then be executed on an aws machine.

In [437]:
import sys
sys.path.insert(0, '../src')
import numpy as np

from frederickson import Frederickson
from environment import Environment
from integersolution import integer_solution
from utils import *
import time

# Import heuristic AHP-mTSP
# Details of the heuristic can be found in "Balanced task allocation by partitioning the
# minmax multiple traveling salesperson" by Isaac Vandermeulen, Roderich Gross, and 
# Andreas Kolling which was published in the proceedings of AAMAS 2019.
sys.path.insert(0, "../ahp_mtsp")
import tsp as ahp

In [438]:
# to save variables on file along to way
# (Notice: the calculations are sometimes very long. We should minimize the risk of losing information)
SAVE_FOLDER = "./saved_results/comparison/"
import os
os.makedirs(SAVE_FOLDER, exist_ok=True)

# the three base modules
little_module = "battle_creekhs_1"
middle_module = "Valleceppi_P0"
big_module = "scuola_3"

# the possible types of structures
# - random: modules are chosen randomly between the three base modules (uniform distribution)
# - decreasing: first third of the modules is composed by big modules, second third by middle
#               ones and the third by the little ones
# - increasing: inverse of "decreasing"
STRUCTURE_TYPES = ["random", "decreasing", "increasing"]

# distance between modules [fixed, for simplicity]
CONNECTION_DISTANCE = 20

In [439]:
def random_module_sequence(types, number_of_modules):
    np.random.seed(0) # seed to ensure reproducibility
    
    # being N = number_of_modules and being len(types) == 3
    # first generate a list of (N//3)*3 modules, just replicating the base modules
    # Notice: the list is not long N unless N % 3 == 0
    modules_list = types * (number_of_modules//len(types))
    
    # Now complete the list adding N % 3 modules (choosing randomly from the possible modules)
    missing_modules = np.random.permutation(types)[:number_of_modules%len(types)]
    modules_list.extend(missing_modules)
    
    # Permutate the modules and get the modules sequence
    modules_sequence = np.random.permutation(modules_list)
    return modules_sequence

def decreasing_module_sequence(types, number_of_modules):
    # initizialize to the last type [little] (then overwrite ONLY the lower parts)
    modules_sequence = [max(types)]*number_of_modules
    section_size = int(np.ceil(number_of_modules/len(types)))
    
    for i in range(min(len(types)-1, number_of_modules)):
        modules_sequence[i*section_size:(i+1)*section_size] = [types[i]]*section_size
    
    return modules_sequence
    
def increasing_module_sequence(types, number_of_modules):
    # initizialize to the first type (then overwrite ONLY the lower parts)
    modules_sequence = [min(types)]*number_of_modules
    section_size = int(np.ceil(number_of_modules/len(types)))
    
    for i in range(min(len(types)-1, number_of_modules)):
        modules_sequence[i*section_size:(i+1)*section_size] = [types[len(types)-1-i]]*section_size
    
    return modules_sequence

In [440]:
big_matrix = get_building_matrix(big_module)
middle_matrix = get_building_matrix(middle_module)
little_matrix = get_building_matrix(little_module)

matrices = [big_matrix, middle_matrix, little_matrix]
types = list(range(len(matrices)))

def env_builder(structure_type, number_of_modules):
    modules_sequence = [max(types)]*number_of_modules
    
    if structure_type == "random":
        modules_sequence = random_module_sequence(types, number_of_modules)
    elif structure_type == "decreasing":
        modules_sequence = decreasing_module_sequence(types, number_of_modules)
    elif structure_type == "increasing":
        modules_sequence = increasing_module_sequence(types, number_of_modules)
    else:
        raise ValueError("Allowed values for structure type are: ", STRUCTURE_TYPES)
    
    distances = [CONNECTION_DISTANCE]*(number_of_modules-1)
    
    env = Environment(matrices, modules_sequence, distances)
    
    return env

# Batch calculations for different scenarios

In [457]:
# temporary. The previous cell contains the values used in the paper
structures = ["random", "decreasing", "increasing"]
robots_numbers = [2,5]
modules_numbers = [5,8]

## DO NOT EXECUTE NEXT CELL
The computation could be extremely slow and the results have been saved in files, so there is no need to recompute them unless you want to change parameters. 

**WARNING**: executing the next cell will OVERWRITE the "dump" files in which the results are stored. Be sure to back that up

In [459]:
for i in range(len(smN)):
    print("Case (structure, number of robots, number of modules):", smN[i])
    print("len fred:", fred_max_len_list[i])
    print("len int:", int_max_len_list[i])
    print("len ahp:", ahp_max_len_list[i])
    print("time fred:", round(fred_time_list[i], 3))
    print("time int:", round(int_time_list[i], 3))
    print("time mat:", round(mat_time_list[i], 3))
    print("time ahp:", round(ahp_time_list[i], 3))
    print()

Case (structure, number of robots, number of modules): ('random', 2, 5)
len fred: 979.07161049
len int: 625.399883555
len ahp: 594.231122327
time fred: 0.499
time int: 0.017
time mat: 0.349
time ahp: 7.584

Case (structure, number of robots, number of modules): ('random', 2, 8)
len fred: 1737.63535742
len int: 1100.03796113
len ahp: 1030.39450106
time fred: 1.351
time int: 0.018
time mat: 0.937
time ahp: 12.739

Case (structure, number of robots, number of modules): ('random', 5, 5)
len fred: 522.564300704
len int: 514.638077579
len ahp: 363.151367205
time fred: 0.51
time int: 0.017
time mat: 0.364
time ahp: 5.413

Case (structure, number of robots, number of modules): ('random', 5, 8)
len fred: 897.014983167
len int: 594.638077579
len ahp: 549.332483467
time fred: 1.334
time int: 0.018
time mat: 0.942
time ahp: 14.921

Case (structure, number of robots, number of modules): ('decreasing', 2, 5)
len fred: 1298.62184213
len int: 749.276155158
len ahp: 689.006255744
time fred: 0.49
time i